<a href="https://colab.research.google.com/github/monica-sabag/Desafio_Extra_Agente_Analise_Dados/blob/main/Desafio_Extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PASSO 1: Instalar e Importar as Bibliotecas
!pip install -q -U google-generativeai

import pandas as pd
import google.generativeai as genai
from google.colab import drive, userdata, files
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import io
import sys
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime
import zipfile
import time
import numpy as np

warnings.filterwarnings('ignore')

# PASSO 2: Conectar o Google Drive e Configurar a API do Gemini
print("Conectando ao Google Drive...")
drive.mount('/content/drive')

print("\nConfigurando a API do Gemini...")
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Chave de API carregada e configurada com sucesso!")
except userdata.SecretNotFoundError as e:
    print(f"Erro: O segredo 'GOOGLE_API_KEY' não foi encontrado. Por favor, adicione-o no cofre de segredos do Colab.")
except Exception as e:
    print(f"Ocorreu um erro ao configurar a API: {e}")

# Variáveis globais para armazenar o DataFrame, o histórico e a UI
df = None
conversation_history = []
output = widgets.Output()
status_label = widgets.Label(value="")
text_box = widgets.Text(description='Pergunta:', placeholder='Digite sua pergunta aqui...')

# --- Configuração do Arquivo de Log ---
caminho_log = '/content/drive/My Drive/Desafio Extra/log_tokens.txt'
def salvar_log_de_tokens(pergunta, tokens_entrada, tokens_saida):
    """Salva os dados de uso de tokens em um arquivo de log."""
    try:
        os.makedirs(os.path.dirname(caminho_log), exist_ok=True)
        with open(caminho_log, 'a') as f:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            f.write(f"Timestamp: {timestamp}\n")
            f.write(f"Pergunta: {pergunta}\n")
            f.write(f"Tokens de Entrada: {tokens_entrada}\n")
            f.write(f"Tokens de Saída: {tokens_saida}\n")
            f.write("-" * 30 + "\n")
    except Exception as e:
        pass

# --- Funções de Lógica de Interação ---

def on_load_drive_clicked(b):
    """Lógica para carregar o arquivo do Google Drive."""
    global df
    global conversation_history
    status_label.value = "Carregando arquivo do Google Drive..."
    with output:
        clear_output()
        caminho_arquivo_zip = '/content/drive/My Drive/Desafio Extra/Kaggle - Credit Card Fraud.zip'
        caminho_arquivo_csv = '/content/drive/My Drive/Desafio Extra/creditcard.csv'
        try:
            if os.path.exists(caminho_arquivo_csv):
                df = pd.read_csv(caminho_arquivo_csv)
            elif os.path.exists(caminho_arquivo_zip):
                print("Arquivo ZIP encontrado. Extraindo...")
                with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
                    extract_dir = os.path.dirname(caminho_arquivo_csv)
                    os.makedirs(extract_dir, exist_ok=True)
                    zip_ref.extract('creditcard.csv', extract_dir)
                df = pd.read_csv(caminho_arquivo_csv)
            else:
                print("Erro: Nenhum dos arquivos ('creditcard.csv' ou 'Kaggle - Credit Card Fraud.zip') foi encontrado.")
                df = None
        except Exception as e:
            print(f"Erro ao carregar o arquivo do Drive: {e}")
            df = None

        if df is not None:
            status_label.value = "Dataset carregado com sucesso!"
            print("\nDataset carregado com sucesso! Agora você pode fazer suas perguntas.")
            conversation_history.clear()
        else:
            status_label.value = "Falha ao carregar o dataset."

def on_upload_clicked(b):
    """Lógica para fazer o upload de um arquivo."""
    global df
    global conversation_history
    status_label.value = "Aguardando upload do arquivo..."
    with output:
        clear_output()
        uploaded_file = files.upload()

        if uploaded_file:
            file_name = next(iter(uploaded_file))
            status_label.value = "Processando upload..."
            with output:
                print(f"\nArquivo '{file_name}' enviado com sucesso!")
                try:
                    if file_name.endswith('.zip'):
                        print("Arquivo ZIP detectado. Extraindo...")
                        with zipfile.ZipFile(io.BytesIO(uploaded_file[file_name]), 'r') as zip_ref:
                            csv_file_name = zip_ref.namelist()[0]
                            df = pd.read_csv(zip_ref.open(csv_file_name))
                        print(f"Arquivo '{csv_file_name}' extraído e carregado com sucesso!")
                    else:
                        df = pd.read_csv(io.BytesIO(uploaded_file[file_name]))
                        print("Dataset carregado com sucesso!")

                    conversation_history.clear()
                    status_label.value = "Dataset carregado com sucesso!"
                except Exception as e:
                    print(f"Erro ao carregar o arquivo: {e}")
                    df = None
                    status_label.value = "Falha no upload do arquivo."
        else:
            with output:
                print("Nenhum arquivo foi enviado. Por favor, faça o upload para começar.")
                status_label.value = "Nenhum arquivo selecionado."


def on_analyze_button_clicked(b):
    """Lógica principal de análise do agente."""
    global df

    # --- Preparação para Captura de Saída (Stdout Redirecionado) ---
    old_stdout = sys.stdout
    redirected_output = io.StringIO()

    with output:
        clear_output(wait=True)
        pergunta_usuario = text_box.value
        text_box.value = ""

        if df is None:
            print("Por favor, carregue um arquivo primeiro.")
            return

        if not pergunta_usuario.strip():
            print("Por favor, digite uma pergunta válida.")
            return

        print(f"\n--- Processando a pergunta: {pergunta_usuario} ---")

        # Adiciona a pergunta do usuário ao histórico da conversa
        conversation_history.append(f"Usuário: {pergunta_usuario}")

        # --- IMPLEMENTAÇÃO DA LIMITAÇÃO DE HISTÓRICO (20 ENTRADAS) ---
        LIMITE_HISTORICO = 20
        history_text = "\n".join(conversation_history[-LIMITE_HISTORICO:])

        # --- PROMPT DO SISTEMA FINALIZADO E OTIMIZADO ---
        sistema_prompt = """
        Você é um assistente de A.E.D. com acesso a um DataFrame do pandas chamado `df`. Sua tarefa é gerar **apenas** código Python para analisar o DataFrame. Não inclua nenhum texto extra, explicação ou marcadores de bloco de código (```python) no início e no final. A saída deve ser apenas o código executável.

        O DataFrame pode variar, mas as colunas de dados são, em geral, numéricas ('float64', 'int64').

        **REGRA DE VISUALIZAÇÃO CRÍTICA:** Para distribuição de **múltiplas variáveis numéricas**, **SEMPRE** utilize `plt.subplot` ou `df.hist()` para agrupar os gráficos em uma única figura. **NÃO gere uma figura separada para cada coluna.**

        **Regra Essencial (Prioridade da Saída):** Se a análise requer uma tabela (ex: describe(), corr(), dtypes), o código DEVE imprimi-la (usando `print()` ou deixando o resultado do pandas ser impresso) ANTES de gerar qualquer gráfico, para que o texto possa ser lido.

        **Reforço de Distribuição:** Para perguntas sobre 'distribuição' ou 'histogramas' e 'padrões temporais', o código deve OBRIGATORIAMENTE calcular e **imprimir** estatísticas relevantes (Assimetria/Curtose/Agregações) para todas as colunas numéricas antes de gerar os gráficos.

        **Regra Essencial (Robustez):** O código DEVE ser robusto. Se for usar uma coluna, verifique se ela existe e, se for para cálculo (média, desvio), inclua `.dropna()` ou trate **apenas** se for estritamente necessário para a execução.

        **Regra Essencial (Tipos de Dados):** Considere colunas que são tecnicamente int64 ou float64, mas que contêm apenas 0 e 1, como **Categóricas Binárias** para propósitos de análise.

        Use as bibliotecas pandas, matplotlib, e seaborn para a análise e visualização.
        Inclua `plt.show()` no final do código se for gerar um gráfico.

        IMPORTANTE: Certifique-se de que o código gerado não contenha NENHUMA linha de texto que não seja código Python (ex: comentários ou notas) e que todas as strings estejam corretamente fechadas.
        """

        # Combina o prompt do sistema, o histórico e a pergunta atual (PARA GERAÇÃO DE CÓDIGO)
        prompt_completo = f"{sistema_prompt}\n\nHistórico da conversa:\n{history_text}\n\nPergunta atual do usuário: {pergunta_usuario}"

        tokens_entrada = 0
        tokens_saida = 0

        try:
            tokens_entrada_obj = genai.GenerativeModel('gemini-2.5-flash').count_tokens(prompt_completo)
            tokens_entrada = tokens_entrada_obj.total_tokens

            modelo = genai.GenerativeModel('gemini-2.5-flash')
            resposta_gemini = modelo.generate_content(prompt_completo)
            codigo_gerado = resposta_gemini.text.strip()

            # 1. Exibir o Código
            print("\nCódigo gerado pelo Gemini:")
            print("```python")
            print(codigo_gerado)
            print("```")

            # 2. Executar e Capturar a Saída
            sys.stdout = redirected_output
            exec(codigo_gerado)

            # 3. Restaurar e Mostrar a Saída
            sys.stdout = old_stdout
            saida_execucao_texto = redirected_output.getvalue()

            # Adiciona a saída de texto da execução ao display (para contexto do usuário)
            if saida_execucao_texto.strip():
                 print("\n--- Saída de Texto do Código Executado ---")
                 print(saida_execucao_texto)

            conversation_history.append(f"Agente: Código executado com sucesso.")

            # --- LÓGICA DE CONCLUSÃO FINAL ---
            print("\n--- Conclusão do Agente ---")

            pergunta_minuscula = pergunta_usuario.lower()

            # --- IDENTIFICADORES DE PERGUNTAS CRÍTICAS ---
            palavras_chave_dtypes = ['tipos de dados', 'numéricos', 'categóricos', 'dtype']
            is_dtypes_question = any(keyword in pergunta_minuscula for keyword in palavras_chave_dtypes)
            palavras_chave_variabilidade = ['variabilidade', 'desvio padrão', 'variância', 'dispersão', 'tendência central', 'média', 'mediana']
            is_variability_question = any(keyword in pergunta_minuscula for keyword in palavras_chave_variabilidade)
            palavras_chave_intervalo = ['intervalo', 'mínimo', 'máximo', 'amplitude']
            is_intervalo_question = any(keyword in pergunta_minuscula for keyword in palavras_chave_intervalo)
            palavras_chave_correlacao = ['influência', 'correlação', 'relacionadas', 'tabelas cruzadas']
            is_correlacao_question = any(keyword in pergunta_minuscula for keyword in palavras_chave_correlacao)

            # --- PROMPT DE CONCLUSÃO COM MEMÓRIA (HISTORY_TEXT) ---
            prompt_conclusao_base = f"""
            A pergunta atual foi: '{pergunta_usuario}'.

            *** HISTÓRICO RECENTE PARA CONTEXTO ***
            {history_text}
            *** FIM DO HISTÓRICO ***

            O código foi executado com sucesso e produziu o seguinte resultado de texto/tabela:
            ---
            {saida_execucao_texto}
            ---

            **REQUISITOS:** Baseado na saída de texto/tabela E no histórico de contexto, gere a conclusão. A saída de texto deve ser a base da sua análise. Não repita informações que já foram analisadas em interações anteriores. Seja direto, objetivo e analítico. Não use blocos de código (` ou ```).
            """

            if is_dtypes_question:
                prompt_conclusao = prompt_conclusao_base + f"""
                **TAREFA:** Baseado na saída, liste as colunas **Numéricas** e a **Categórica Binária** ('Class'). Seja direto e conciso.
                """
            elif is_variability_question:
                prompt_conclusao = prompt_conclusao_base + f"""
                **TAREFA:** Analise a tabela de estatísticas descritivas na seção de saída. Comente sobre as variáveis que possuem a **maior e a menor dispersão** e as principais **medidas de tendência central (média e mediana)**. **NÃO** peça pela Média/Mediana se já estiver no histórico.
                """
            elif is_intervalo_question:
                prompt_conclusao = prompt_conclusao_base + f"""
                **TAREFA:** Focar APENAS na análise da TABELA de Mínimo e Máximo na seção de saída. Comente a amplitude da coluna **Time** e **Amount**, e o comportamento das colunas **V-X**. Seja **extremamente conciso**.
                """
            elif is_correlacao_question:
                prompt_conclusao = prompt_conclusao_base + f"""
                **TAREFA:** Focar APENAS na análise da TABELA (ou heatmap) de correlação na seção de saída. Comente as **correlações positivas e negativas mais fortes** e as **variáveis com correlação mais próxima de zero**. Seja **extremamente conciso**.
                """
            else:
                # --- ANÁLISE INTEGRADA (GRÁFICO) ---
                prompt_conclusao = prompt_conclusao_base + f"""
                **TAREFA (INTEGRADA):** A análise gerou um GRÁFICO e SAÍDA DE TEXTO. Priorize a análise do **texto (tabelas/estatísticas)** para quantificar padrões (assimetria, curtose, agregados). Use a **visualização** (histogramas, scatter plots) para complementar o texto, descrevendo **padrões, tendências ou agrupamentos visíveis** (ex: bimodais, concentração, outliers). Seja **extremamente conciso** (máximo 4 frases), focando na **integração** dos dados e do visual.
                """
                # --- FIM ---

            resposta_conclusao = modelo.generate_content(prompt_conclusao)
            conclusao_texto = resposta_conclusao.text
            print(conclusao_texto)

            # Adiciona a CONCLUSÃO ao histórico (para ser usada na próxima iteração)
            conversation_history.append(f"Agente: {conclusao_texto}")

            tokens_saida_obj = genai.GenerativeModel('gemini-2.5-flash').count_tokens(conclusao_texto)
            tokens_saida = tokens_saida_obj.total_tokens

        except Exception as e:
            sys.stdout = old_stdout
            print(f"\nErro na execução ou comunicação: {e}")
            print("Não foi possível gerar ou executar o código. Tente reformular a sua pergunta.")
            conversation_history.append(f"Agente: Erro na execução do código. Não foi possível gerar ou executar. Tente reformular a sua pergunta.")

        salvar_log_de_tokens(pergunta_usuario, tokens_entrada, tokens_saida)

# --- Layout dos Widgets da Interface ---

print("Bem-vindo ao Agente de Análise de Dados!")

# Cria os botões
button = widgets.Button(description="Analisar")
load_drive_button = widgets.Button(description="Carregar do Google Drive", button_style='success')
upload_button_display = widgets.Button(description="Fazer Upload de CSV", button_style='info')

# Conecta os botões às funções
load_drive_button.on_click(on_load_drive_clicked)
upload_button_display.on_click(on_upload_clicked)
button.on_click(on_analyze_button_clicked)

# Monta o layout da interface
header = widgets.HTML("<h3>Faça sua Pergunta</h3>")
text_and_button_box = widgets.HBox([text_box, button])
load_buttons_box = widgets.HBox([load_drive_button, upload_button_display])
status_box = widgets.VBox([status_label])

# Exibe todos os widgets
display(header, text_and_button_box, load_buttons_box, status_box, output)